# Day 9: Disk Fragmenter

## Import libraries

In [1]:
import copy

## Import data

In [2]:
# *** [IMPORT DATA] ***
# NOTE: In the given puzzle input:
# - A single string line that represents a disk map.
# - The disk map uses a dense format to represent the layout of files and free space on the disk.
# - The digits alternate between indicating the *length of a file* and the *length of free space*.
# - E.g. A disk map = '12345' represents: 1-block file; 2 blocks of free space; 3-block file; 4-blocks of free space; 5-block file.
# =====================================================================================================================
# ! Open the file for reading mode (= default mode if the mode is not specified)
file = open("../data/24_day-9_input-test.txt", "r") 

# Read all the data in the file
file_data = file.read().strip()

print(file_data)
# ====================================================================================================================

2333133121414131402


## Helper functions

In [3]:
def rearrange_disk_map_p1(s):
    left = 0  # Start from the beginning of the string
    right = len(s) - 1  # Start from the end of the string

    while left < right:
        # If the current character is a '.', find the first number from the right
        if s[left] == '.':
            # Traverse from the right to find the first number
            while right > left and s[right] == '.':
                right -= 1
                
            # Swap the '.' with the number
            if right > left:
                s[left], s[right] = s[right], s[left]
                right -= 1

        left += 1 # END_WHILE

    # Return the rearranged string
    return s

In [5]:
def rearrange_disk_map_p1_alt(disk_map):
    # Parse the disk map into blocks
    blocks = []
    file_id = 0
    is_file = True  # starts with file
    i = 0
    
    while i < len(disk_map):
        length = int(disk_map[i])

        if is_file:
            blocks.extend([str(file_id)] * length)
            file_id += 1
        else:
            blocks.extend(['.'] * length)

        is_file = not is_file
        i += 1
    
    # Simulate compaction
    changed = True

    while changed:
        changed = False
        # Find the first free space
        first_free = None

        for pos in range(len(blocks)):
            if blocks[pos] == '.':
                first_free = pos
                break

        if first_free is None:
            break  # no free space left

        # Find the last file block to move
        last_file_pos = None

        for pos in range(len(blocks) - 1, -1, -1):
            if blocks[pos] != '.':
                last_file_pos = pos
                break

        if last_file_pos is not None and last_file_pos > first_free:
            # Move the block
            blocks[first_free] = blocks[last_file_pos]
            blocks[last_file_pos] = '.'
            changed = True
    
    return blocks
    # # Calculate checksum
    # checksum = 0
    
    # for pos in range(len(blocks)):
    #     if blocks[pos] != '.':
    #         checksum += pos * int(blocks[pos])
    
    # return checksum

In [ ]:
def rearrange_disk_map_p2(disk_map):
    # Parse the disk map into blocks
    blocks = []
    file_id = 0
    is_file = True  # starts with file
    i = 0

    while i < len(disk_map):
        length = int(disk_map[i])

        if is_file:
            blocks.extend([str(file_id)] * length)
            file_id += 1
        else:
            blocks.extend(['.'] * length)

        is_file = not is_file
        i += 1
    
    total_files = file_id

    # Process files in reverse order of file_id
    for current_file in range(total_files - 1, -1, -1):
        current_file_str = str(current_file)
        # Find the current file's positions
        file_positions = [i for i, block in enumerate(blocks) if block == current_file_str]
        
        if not file_positions:
            continue
        
        file_length = len(file_positions)

        # Find the leftmost contiguous free space that can fit the file
        best_start = None
        best_end = None
        
        # Search for a free space segment before the first occurrence of the file
        first_file_pos = file_positions[0]
        free_start = None

        for pos in range(first_file_pos):
            if blocks[pos] == '.':
                if free_start is None:
                    free_start = pos
            else:
                if free_start is not None:
                    free_length = pos - free_start

                    if free_length >= file_length:
                        best_start = free_start
                        best_end = free_start + file_length
                        break

                    free_start = None
        else:
            if free_start is not None:
                free_length = first_file_pos - free_start

                if free_length >= file_length:
                    best_start = free_start
                    best_end = free_start + file_length
        
        if best_start is not None:
            # Move the file to the new position
            # Remove the file from current positions
            for pos in file_positions:
                blocks[pos] = '.'
            
            # Place the file in the new positions
            for pos in range(best_start, best_end):
                blocks[pos] = current_file_str
    
    # Calculate checksum
    checksum = 0

    for pos in range(len(blocks)):
        if blocks[pos] != '.':
            checksum += pos * int(blocks[pos])
            
    return checksum

## Part 1

In [ ]:
# *** [PART 1] ***
# ! PROBLEM: The amphipod would like to move file blocks *one at a time* from the END of the disk to the LEFTMOST free space block (until there are no gaps remaining between file blocks - see examples on website).
# - Each file on disk also has an ID number based on the *order* of the files as they appear BEFORE they are rearranged, starting with ID: 0.
# - E.g. A disk map '12345' has 3 files: 1-block file (ID: 0); 3-block file (ID: 1); 5-block file (ID: 2).
# - - 'x' ID digits are used to represent EACH block (where 'x' = file block number) & '.' = free space.
# - E.g. A disk map '12345' becomes '0..111....22222' after rearrangement.
# - TODO: Calculate the resulting filesystem checksum of the RE-ARRANGED & MOVED disk map: Add up the result of multiplying each of the blocks' position with the file ID number it contains.
# - E.g. '0099811188827773336446555566...' = '(0 * 0 = 0) + (1 * 0 = 0) + (2 * 9 = 18) + (3 * 9 = 27) + (4 * 8 = 32) + ... ' (Multiply array item idx by value)
# ====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated in repeated runs.
disk_map = copy.deepcopy(file_data)
arrTransformedDiskMap = []
rearrangedDiskMap = ''
blockCounter = 0
checkSum = 0

for i in range(len(disk_map)):
    if i % 2 == 0: # If the index is even (i.e., we're looking at a block file)
        for j in range(int(disk_map[i])): #'x' ID digits are used to represent EACH block (where 'x' = file block number)
            # NOTE: DO NOT use 'transformedDiskMap +=' because if 'blockCounter' number > 1 digit, then when transform string to a list, it will break up all numbers in the string into singluar digits (E.g. '10' => '1','0'), therefore append each number AS A WHOLE into an array list
            arrTransformedDiskMap.append(str(blockCounter))
            
        blockCounter += 1
    elif i % 2 != 0: # If the index is odd (i.e., we're looking at a free block space)
        for j in range(int(disk_map[i])):
            arrTransformedDiskMap.append('.') # Append '.' as many times as the CURRENT free block space number

""" Re-arrange transformed disk map """
# Move file blocks (ID numbers) from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks)
rearrangedDiskMap = rearrange_disk_map_p1(arrTransformedDiskMap)
# rearrangedDiskMap2 = rearrange_disk_map_p1_alt(disk_map)
print(rearrangedDiskMap)
# print(rearrangedDiskMap2)

for i in range(len(rearrangedDiskMap)):
    if rearrangedDiskMap[i] != '.':
        checkSum += (i * int(rearrangedDiskMap[i]))

print("Filesystem checksum (Part 1):", checkSum)
# ====================================================================================================================

['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7', '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5', '6', '6', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
['0', '0', '9', '9', '8', '1', '1', '1', '8', '8', '8', '2', '7', '7', '7', '3', '3', '3', '6', '4', '4', '6', '5', '5', '5', '5', '6', '6', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.', '.']
Filesystem checksum (Part 1): 1928


## Part 2

In [ ]:
# *** [PART 2] ***
# ! PROBLEM: Upon completion, two things immediately become clear. First, the disk definitely has a lot more contiguous free space, just like the amphipod hoped. Second, the computer is running much more slowly! Maybe introducing all of that file system fragmentation was a bad idea?
# - TODO: Rather than moving individual blocks, compact disk files by moving WHOLE files instead.
# - Attempt to move *whole files* to the *leftmost span* of free space blocks that could fit the file.
# - Attempt to move each file *exactly once* in order of *decreasing file ID number*, starting with the file with the *highest file ID number*.
# - If there is no span of free space to the left of a file that is large enough to fit the file, the file does not move.
#====================================================================================================================
# ! Create a deep (independent) copy of the data, such that changes made to the copy do not affect the original data to still test/re-run Part 1/2 with the correct INITIAL (and not modified) data
# - NOTE: Not using a deep copy will modify the original data after running Part 1/2, therefore no correct output will be calculated in repeated runs.
disk_map = copy.deepcopy(file_data)
arrTransformedDiskMap_p2 = []
rearrangedDiskMap_p2 = ''
blockCounter_p2 = 0
checkSum_p2 = 0

for i in range(len(disk_map)):
    if i % 2 == 0: # If the index is even (i.e., we're looking at a block)
        for j in range(int(disk_map[i])): #'x' ID digits are used to represent EACH block (where 'x' = file block number)
            # NOTE: DO NOT use 'transformedDiskMap +=' because if 'blockCounter' number > 1 digit, then when transform string to a list, it will break up all numbers in the string into singluar digits (E.g. '10' => '1','0'), therefore append each number AS A WHOLE into an array list
            arrTransformedDiskMap_p2.append(str(blockCounter_p2))
            
        blockCounter_p2 += 1
    elif i % 2 != 0: # If the index is odd (i.e., we're looking at a free space)
        for j in range(int(disk_map[i])):
            arrTransformedDiskMap_p2.append('.') # Append '.' as many times as the CURRENT free space number

# """ Re-arrange transformed disk map """
# # Move file blocks (ID numbers) from the end of the disk to the leftmost free space block (until there are no gaps remaining between file blocks)
# rearrangedDiskMap_p2 = rearrange_disk_map_p2(arrTransformedDiskMap_p2)
# # print(rearrangedDiskMap_p2)

# for i in range(len(rearrangedDiskMap_p2)):
#     if rearrangedDiskMap_p2[i] != '.':
#         checkSum_p2 += (i * int(rearrangedDiskMap_p2[i]))

# print("Filesystem checksum (Part 2):", checkSum_p2)



# # Test the function
# input_string = '00...111...2...333.44.5555.6666.777.888899'
# expected_output = '00992111777.44.333....5555.6666.....8888..'

# output = swap_dot_with_digits(input_string)
# print("Output:", output)
# print("Matches expected output:", output == expected_output)

